In [2]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a05_unidades_de_apoyo as ua
from proyeccion_rdr.produccion.a04_ambulatorio import (
    leer_consultas_medicas,
    leer_consultas_no_medicas,
)


pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [ ]:
# Define las rutas de los archivos
ruta_raw = "../data/raw"

# Define las rutas de la base de CAE y GRD
ruta_cae = ruta_raw + "6_ambulatorio/df_procesada_consultas.csv"
ruta_grd = ruta_raw + "7_egresos_grd/df_procesada_grd.csv"

# Define las rutas de los archivos de apoyo diagnostico
ruta_apoyo = ruta_raw + "8_unidades_de_apoyo/"

ruta_laboratorio = ruta_apoyo + "df_procesada_laboratorio_clinico.csv"
ruta_farmacia = ruta_farmacia_amb = ruta_apoyo + "df_procesada_farmacia.csv"
ruta_imagenologia = ruta_apoyo + "df_procesada_imagenologia.csv"
ruta_umt = ruta_apoyo + "df_procesada_umt.csv"